#### Step 0

0. defining location
1. defining peak power and technology type
2. define TRACKING_TYPE (fixed, single axis, two axes)
3. define RACKING_MODEL
4. define TILT (used both for fixed modules and single axis tracking)
5. define AZIMUTH (used both for fixed modules and single axis tracking)
6. define the surface type

based on such information, the user should be able to:

7. choose the module
8. choose the inverter

1) Goals & site

Location: Latitude/Longitude (map pin), auto-fill altitude (pvgis).
Target peak power (kWp) and Max area (m²) (optional).
Tech type (dropdown): TechChoice.
Tracking type (dropdown): TrackingType.
Mounting place (dropdown): RackingType
Azimuth (dial/compass) & Tilt (slider).
Surface type (dropdown → sets albedo): SurfaceType

In [1]:
from pvlib.pvsystem import retrieve_sam

from common.enums import RackingType, SurfaceType, TechChoice, TrackingType
from models.simulatorrecs.pv_plant import PVPlant

# input after selecting kwp, lat, lon, tech_choice
# TODO: substitute with EnergyPlantEditInfoContainer
# TODO: evaluate possibility of the creation of a new model and endpoint for creation --> CablingSchemeEditInfoContainer
pv_plant = PVPlant(
    uri='',
    pod='IT000E00092',
    pod_uri='paidsfhgapghadg',
    kwp=1000,
    maximum_area=0,
    latitude=41.9028,
    longitude=12.4964,
    angle=21.2,
    aspect=0,
    tech_choice=TechChoice.Mono_c_Si,
    tracking_type=TrackingType.fixed,
    mounting_place=RackingType.semi_integrated,
    surface_type=SurfaceType.concrete,
    percentage_loss=17,
    yearly_loss=0.5,
)

In [2]:
from apis.pvlib_helper.geographical_helper import GeographicalHelper
from apis.pvlib_helper.pvsystems_helper import PVSystemHelper


pv_system_helper = PVSystemHelper()
cecmodules = retrieve_sam('cecmod')
inverters = retrieve_sam('cecinverter')
u_c, u_v = RackingType.get_default_model_temperature_params(pv_plant.mounting_place)
altitude = GeographicalHelper.get_elevation_opentopo(lat=pv_plant.latitude, lon=pv_plant.longitude)
hot_temperature, cold_temperature = GeographicalHelper.get_extreme_ambient_temperatures(
    lat=pv_plant.latitude, 
    lon=pv_plant.longitude, 
    altitude=altitude,
    surface_type=pv_plant.surface_type.name if pv_plant.surface_type is not None else None,
    racking_type=pv_plant.mounting_place.name,
    tilt_angle=pv_plant.angle
    )
filtered_modules = pv_system_helper.get_cec_modules_by_tech_choice(pv_plant.get_tech_choice_description)
filtered_modules = pv_system_helper.filter_legacy_or_out_of_scope_cec_modules(filtered_modules)
filtered_modules = pv_system_helper.filter_cec_modules_by_racking_type(filtered_modules, pv_plant.mounting_place)
filtered_inverters = pv_system_helper.get_cec_inverters_by_kwp(pv_plant.kwp)
filtered_modules, filtered_inverters = pv_system_helper.filter_cec_modules_by_voltage(
    filtered_modules.T,
    filtered_inverters,
    u_v=u_v,
    u_c=u_c,
    hot_temperature_ambient=hot_temperature,
    cold_temperature_ambient=cold_temperature
)


In [3]:
from models.simulatorrecs.pv_plant import InverterBasicInfo, ModuleBasicInfo


pv_plant.set_altitude(altitude)
pv_plant.set_modules([ModuleBasicInfo(name=name, nominal_power=m['STC']) for name, m in filtered_modules.T.iterrows()])
pv_plant.set_inverters([InverterBasicInfo(name=name, nominal_power=i['Paco']) for name, i in filtered_inverters.T.iterrows()])

##### User chooses either a pv module OR an inverter

the app must, then, filter again the dropdown of the technology that the user has not picked yet. The procedure is the same for the two choices, since the method pv_system_helper.filter_cec_modules_by_voltage deals with it.

In [4]:
picked_inverter = pv_plant.inverter_names[0]
pv_plant.inverter_name = picked_inverter.name
picked_inverter

InverterBasicInfo(name='AEG_Power_Solutions__Protect_MPV_075_01__480V_', nominal_power=75000.0)

In [5]:
import pandas as pd

filtered_inverter = pd.DataFrame(filtered_inverters[picked_inverter.name])

filtered_modules, filtered_inverters = pv_system_helper.filter_cec_modules_by_voltage(
    filtered_modules.T,
    filtered_inverter,
    u_v=u_v,
    u_c=u_c,
    hot_temperature_ambient=hot_temperature,
    cold_temperature_ambient=cold_temperature
)
pv_plant.set_modules([ModuleBasicInfo(name=name, nominal_power=m['STC']) for name, m in filtered_modules.T.iterrows()])
pv_plant.set_inverters([InverterBasicInfo(name=name, nominal_power=i['Paco']) for name, i in filtered_inverters.T.iterrows()])

In [6]:
picked_module = pv_plant.module_names[0]
pv_plant.module_name = picked_module.name

#### Step 1

0. choosing the number of modules in series (n_series)
1. choosing the number of series in parallel (n_strings)
2. suggest an ideal number of inverters
3. choose number of inverters
4. output the actual peak power achieved
5. the app must tell the user when the configuration he is forcing is not feasible

In [7]:
module = pv_plant.get_module_parameters
inverter = pv_plant.get_inverter_parameters

In [8]:
temperature_model_parameters = pv_plant.get_temperature_model_parameters
loc = pv_plant.get_location

In [9]:
n_series, n_strings, sizing = pv_system_helper.pick_stringing(
    module, 
    inverter,
    t_cell_cold=PVSystemHelper._estimate_cell_T(cold_temperature, Uc=u_c, Uv=u_v), 
    t_cell_hot=PVSystemHelper._estimate_cell_T(hot_temperature, Uc=u_c, Uv=u_v), 
    dc_ac_ratio=1.20
)

In [10]:
print("Picked:", n_series, "modules per string,", n_strings, "strings")
print("Sanity:", sizing)

Picked: 16 modules per string, 19 strings
Sanity: {'mppt_low': 305.0, 'mppt_high': 595.0, 'vdcmax': 595.0, 'idcmax': 192.022126, 'paco': 75000.0, 'Vmp_cold': 28.972975782978416, 'Vmp_hot': 19.66141176844082, 'Voc_cold': 34.849126317019, 'n_series_min_by_mppt': 16, 'n_series_max_by_mppt': 20, 'n_series_max_by_voc': 16, 'chosen_series': 16, 'Pmp_module_stc': 290.157, 'Pmp_string_stc': 4642.512, 'dc_target_W': 90000.0, 'chosen_strings': 19, 'total_imp_hot': 163.32237878098036}


In [11]:
pv_plant.set_modules_per_string(n_series)
pv_plant.set_strings(n_strings)
pv_plant.set_number_of_inverters()

In [12]:
pv_plant.effective_peak_power_dc

998.14008

In [ ]:
pv_plant.effective_peak_power_ac

848.6842105263158

In [ ]:
pv_plant.dcac_ratio

#### Step 2

4. define MAX_ANGLE if it is single axis tracking
5. define BACKTRACK if it is single axis tracking (True vs False)
6. define GCR if it is single axis tracking (default 0.35)
7. define temperature parameters (rules for default parameters is defined in class RackingType)
8. define b0 parameter of the module for IAM coefficient (defulat 0.05) for the ashrae aoi model
0. define wiring material (WiringMaterial enum)
1. define wiring length
2. define wiring cross section
3. define AC_WIRING_FACTOR
4. define AVAILABILITY
5. define LID_FACTOR
6. define MODULE_QUALITY_FACTOR
7. define MISMATCH_FACTOR
8. extract SOILING_MONTHLY (Monthly soiling transmittance)

#### Step 3

2. fetch and preprocess (extract ghi, dhi, dni, airmass, precipitable_water) meteo data

Step 5

0. define pvlib.pvsystem.Array
1. define pvlib.pvsystem.PVSystem
2. define pvlib.modelchain.ModelChain
3. apply posterior loss model
4. multiply by the number of inverters

#### Step 6

0. Apply Markov Chain Montecarlo model to the 12 years of historical backcast of production
1. Save the ProductionMatrix
2. Simulate self consumption and grid injection